## В этом домашнем задании вы сделаете первые шаги в мире линейной бинарной классификации!

In [156]:
import pandas as pd
import numpy as np
import warnings

#### Задание 1

Мы будем работать с данными **Microsoft Malware Detection**

Таргетом будет последний столбец `HasDetection`, который принимает значения $\{0,\, 1\}$ в случае отсутствия или наличия вируса на компьютере соответственно. Признаками будут выступать всевозможные характеристики компьютера.

In [157]:
data = pd.read_csv('../data/train.csv')

C:\Users\danil\AppData\Local\Temp\ipykernel_9876\425167489.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/train.csv')


In [158]:
data.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1


In [159]:
data.shape

(200000, 83)

Удалите константные признаки и признаки `ProductName` `MachineIdentifier`

In [160]:
### Your code is here

data = data.drop(columns=['ProductName', 'MachineIdentifier'], axis=1)


Посмотрите на соотношение классов в таргете. Все ли хорошо?

In [161]:
print(sum(data['HasDetections'] == 1), '- positive class,')
print(sum(data['HasDetections'] == 0), '- negative class')

100060 - positive class,
99940 - negative class


Ответьте на вопрос: почему с вашей точки зрения важно иметь представление о балансе классов в ваших данных?

Избавьтесь от пропусков в данных! 

Новый для нас прием: признаки с более чем половиной пропусков следует удалить.

Согласитесь, если в вашей колонке среди 100 объектов всего лишь у 2 есть какое-то непропущенное значение, странно все остальные заполнять средним от этих двух чисел. Такие "редкие" признаки лучше вообще опустить!


В категориальных колонках заменим отсутствующую категорию просто некоторой новой и назовем ее `NaN`.

А в числовых, ради разнообразия, заполним пропуски медианным значением.

In [162]:
### Your code is here

numeric_columns = data.select_dtypes(exclude=[np.object_]).columns
data[numeric_columns].head(2)

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0,7.0,0,NaN,53447.0,1.0,1.0,1,29,128035.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,0,7.0,0,NaN,53447.0,1.0,1.0,1,93,1482.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0


In [163]:
data[numeric_columns].describe()

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,200000.0,199236.000000,200000.000000,9703.000000,199196.000000,199196.000000,199196.000000,200000.00000,200000.000000,192785.000000,...,196414.000000,200000.000000,73161.0,199652.000000,200000.000000,200000.000000,198379.000000,193263.000000,193263.000000,200000.000000
mean,0.0,6.848175,0.017090,1654.631969,47867.326116,1.324916,1.021140,0.98768,107.989305,81288.197287,...,33015.526770,0.485065,0.0,0.007022,0.125465,0.037850,0.057249,0.282589,7.877649,0.500300
std,0.0,1.016713,0.129607,996.743491,14003.576777,0.521275,0.166286,0.11031,62.960237,48928.959786,...,21220.733321,0.499778,0.0,0.083504,0.331246,0.190834,0.232318,0.450259,4.555336,0.500001
min,0.0,0.000000,0.000000,1.000000,39.000000,1.000000,0.000000,0.00000,1.000000,7.000000,...,5.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.0,7.000000,0.000000,788.000000,49480.000000,1.000000,1.000000,1.00000,51.000000,36825.000000,...,13156.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,0.0,7.000000,0.000000,1632.000000,53447.000000,1.000000,1.000000,1.00000,97.000000,82373.000000,...,33070.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000
75%,0.0,7.000000,0.000000,2373.000000,53447.000000,2.000000,1.000000,1.00000,160.000000,123975.000000,...,52436.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,11.000000,1.000000
max,0.0,8.000000,1.000000,3204.000000,70486.000000,5.000000,4.000000,1.00000,222.000000,167962.000000,...,72091.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000,1.000000


In [164]:
categorical_columns = data.select_dtypes(include=[np.object_]).columns
data[categorical_columns].head(2)

,EngineVersion,AppVersion,AvSigVersion,Platform,Processor,OsVer,OsPlatformSubRelease,OsBuildLab,SkuEdition,PuaMode,...,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing
0,1.1.15100.1,4.18.1807.18075,1.273.1735.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,NaN,...,10.0.17134.165,amd64,rs4_release,Professional,PROFESSIONAL,UUPUpgrade,UNKNOWN,IS_GENUINE,Retail,Retail
1,1.1.14600.4,4.13.17134.1,1.263.48.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,NaN,...,10.0.17134.1,amd64,rs4_release,Professional,PROFESSIONAL,IBSClean,UNKNOWN,OFFLINE,Retail,NOT_SET


In [165]:
### Отброс константных признаков

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[numeric_columns])

cutter = VarianceThreshold(threshold=0)  # Порог установлен на 0 для поиска полностью константных параметров
cutter.fit(scaled_data)

selected_features = cutter.get_support()  # Булевый массив выбранных признаков / False - означает, что параметр константный или квазиконстантный
constant_cols = [col for col, selected in zip(numeric_columns, selected_features) if not selected]

data = data.drop(columns=constant_cols, axis=1)
numeric_columns = numeric_columns.drop([col for col in constant_cols if col in numeric_columns])

In [166]:
# Удаляем признаки с более чем половиной пропусков

columns_to_drop = [data.isna().sum().index[index] for index, row in enumerate(data.isna().sum()) if data.isna().sum().iloc[index] > data.shape[0] / 2]
data = data.drop(columns=columns_to_drop, axis=1)


In [167]:
numeric_columns = numeric_columns.drop([col for col in columns_to_drop if col in numeric_columns])
len(numeric_columns)

48

In [168]:
categorical_columns = categorical_columns.drop([col for col in columns_to_drop if col in categorical_columns])
len(categorical_columns)

25

In [169]:
### Заполняем пропуски в числовых признаках медианой
for col in numeric_columns:
    mean = data[col].mean()
    data[col] = data[col].fillna(mean) 

In [170]:
### Заполняем пропуски в категориальных признаках категорией "NaN"
for col in categorical_columns:
    data[col] = data[col].fillna("NaN")

Создайте копию полученного датафрейма и положите ее в переменную data_2. Понадобится в следующих заданиях.

In [184]:
data_2 = data.copy()

In [185]:
data_2.shape

(200000, 73)

Так же поработаем над всеми категориальными колонкам перед запуском непосредственно моделей.

Провернем самый базовый и наглый метод - несмотря на количество уникальных значений в каждой категории, просто применим ко всей категориальной части датасета `OneHotEncoding`

In [18]:
### Your code is here

data[categorical_columns].describe(include=object)

for col in categorical_columns:
    one_hot = pd.get_dummies(data[col], prefix=col, drop_first=True)
    data = pd.concat((data.drop(col, axis=1), one_hot), axis=1)

data

,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,...,Census_ActivationChannel_Retail,Census_ActivationChannel_Retail:TB:Eval,Census_ActivationChannel_Volume:GVLK,Census_ActivationChannel_Volume:MAK,Census_FlightRing_NOT_SET,Census_FlightRing_RP,Census_FlightRing_Retail,Census_FlightRing_Unknown,Census_FlightRing_WIF,Census_FlightRing_WIS
0,7.0,0,53447.0,1.0,1.0,1,29,128035.0,18.000000,35.0,...,True,False,False,False,False,False,True,False,False,False
1,7.0,0,53447.0,1.0,1.0,1,93,1482.0,18.000000,119.0,...,True,False,False,False,True,False,False,False,False,False
2,7.0,0,53447.0,1.0,1.0,1,86,153579.0,18.000000,64.0,...,False,False,False,False,False,False,True,False,False,False
3,7.0,0,53447.0,1.0,1.0,1,88,20710.0,24.870919,117.0,...,False,False,False,False,False,False,True,False,False,False
4,7.0,0,53447.0,1.0,1.0,1,18,37376.0,24.870919,277.0,...,True,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.0,1,3371.0,2.0,1.0,1,160,14821.0,18.000000,204.0,...,False,False,False,False,False,False,True,False,False,False
199996,7.0,0,53447.0,1.0,1.0,1,60,86819.0,18.000000,240.0,...,False,False,False,False,False,False,True,False,False,False
199997,7.0,0,62773.0,1.0,1.0,1,9,78912.0,27.000000,167.0,...,False,False,False,False,False,False,True,False,False,False
199998,7.0,0,49480.0,2.0,1.0,1,158,87401.0,18.000000,230.0,...,False,False,False,False,False,False,True,False,False,False


###  Разделим выборку на тренировочную и тестовую

P.S. в задачах классификации, как и в задаче регрессии, можно использовать технологию Кросс-Валидации. Например, по одному из двух следующих сценариев:

1) Отделить валидацию и тест, произвести подбор лучшей модели с помощью `K-Fold` на валидации, финально обучить выбранную модель на всей валидации и замерить качество на заранее отложенном финальном тесте!

2) Всю выборку назвать валидационной и на ней применить `K-Fold` без финального замера.

В этой домашней работе попросим Вас быть еще проще! :)
Реализуем просто технологию отложенной выборки в пропорции 3:1

In [19]:
from sklearn.model_selection import train_test_split 

X = data.drop(columns=['HasDetections'])
y = data['HasDetections']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)

Соберите `Pipeline`, реализовав в нем 2 шага: стандартизация данных через `StandardScaler` и обучение логистической регрессии с помощью `LogisticRegression`, положите результаты в переменную `pipe`, а в классе модели `LogisticRegression` укажите параметр `penalty='none'`

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

### Your code is here

pipe = Pipeline([('scaler', StandardScaler()),
                 ('LR', LogisticRegression(penalty=None))])

Чтобы замерить качество работы такой модели на трейне и на тесте воспользуемся функцией `cross_validate`

Вопрос: что передавать ей в параметр cv? Ведь мы уже разделили нашу выборку на трейн и тест (имеем всего 1 fold). Для этого можно просто передать список от кортежа, содержащего индексы тренировочных и тестовых объектов.

In [21]:
from sklearn.model_selection import cross_validate
import datetime

custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X, y, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

E:\Рабочий стол С\python_projects\carpov_courses_python\machine_learning\less_10\venv\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "E:\Рабочий стол С\python_projects\carpov_courses_python\machine_learning\less_10\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "E:\Рабочий стол С\python_projects\carpov_courses_python\machine_learning\less_10\venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "E:\Рабочий стол С\python_projects\carpov_courses_python\ma

Accuracy на трейне: nan
Accuracy на тесте: 0.622
Время работы алгоритма: 0:04:45.983041


Что можете сказать про время работы алгоритма?

Очевидно, оно достаточно большие. Уж тем более для линейных моделей.

Такое как раз-таки происходит из-за того, что количество фичей, который мы передали нашей модели - гигантское! Классу требуется много времени и памяти, чтобы обработать датасет.

Поэтому те колонки, в которых количество уникальных категорий превышает какое-то адекватное число, следует кодировать иначе, нежели с помощью технологии `One-Hot-Encoding`.

Теперь вы верите, что более умные кодировки зачастую прям необходимы! Раньше мы этот факт не демонстрировали!

Дело еще вот в чем: в классе `LogisticRegression`, как и, например, `Lasso`, есть параметр, ограничивающий максимальное количество итераций во время обучения модели. Так, если данных много и итераций тоже ожидается большое число, найденная разделяющая гиперплоскость может оказаться не самой лучшей, так как наш алгоритм (будь то градиентный спуск или любой иной) просто 'не доползет'. 

#### Задание 2

Теперь попробуем другой метод кодирования категориальных колонок, а именно счётчики.
Построем ту же модель и на том же разделении, просто заново иначе переобработаем датасет. 

Для тех категориальных признаков, у которых количество уникальных значений в колоночках больше 5, применим `MeanTargetEncoding`.

Для остальных оставим любимый `OneHotEncoding` (как делали на практике и в предыдущем уроке).

In [186]:
### Your code is here

categorical_mean_columns = []

for col in categorical_columns:

    if data_2[col].nunique() > 5:
        mean_grouped = data_2.groupby(col)['HasDetections'].mean()
        data_2[col] = data_2[col].map(mean_grouped)
        categorical_mean_columns.append(col)

    else:
        one_hot = pd.get_dummies(data_2[col], prefix=col, drop_first=True)
        one_hot.replace({False: 0, True: 1}, inplace=True)
        data_2 = pd.concat((data_2.drop(col, axis=1), one_hot), axis=1)

C:\Users\danil\AppData\Local\Temp\ipykernel_9876\3700250308.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  one_hot.replace({False: 0, True: 1}, inplace=True)
C:\Users\danil\AppData\Local\Temp\ipykernel_9876\3700250308.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  one_hot.replace({False: 0, True: 1}, inplace=True)
C:\Users\danil\AppData\Local\Temp\ipykernel_9876\3700250308.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitl

In [188]:
X_2 = data_2.drop(columns=['HasDetections'])
y_2 = data_2['HasDetections']

Опять обучим модель, пока что без изменений! Скажите, стало ли быстрее? А что с качеством?

In [176]:
### Your code is here

X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)

custom_cv = [(X_2_train.index.to_list(), X_2_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X_2, y_2, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

Accuracy на трейне: 0.638
Accuracy на тесте: 0.638
Время работы алгоритма: 0:00:02.639755


#### Задание 3: Регуляризация

Как и в моделях регрессии, решая задачу классификации, можем штрафовать минимизируемый функционал за большие веса, добавив к нему L1 или L2 норму весов (все как раньше!).

Для этого в изначальном классе `LogisticRegression` изменить параметр `penalty` на l1 или l2 соответственно. Выберите второй вариант! Можно воспользоваться методом `set_params` и применить его к `pipe`.

In [177]:
### Your code is here

LR_model = pipe.named_steps['LR']
LR_model.set_params(penalty='l2')

LogisticRegression(C=0.01)

Теперь наша модель будет строить логистическую регрессию с L2 регуляризатором! Помним, что у регуляризируемых моделей есть гиперпараметр, контролирующий силу регуляризации, который выбирается ДО запуска метода fit, то есть заранее, когда модель еще не обучена. 

Конечно же, выбор этого параметра влияет итоговые результаты. Хочется поперебирать различные параметры регуляризации и найти такой, при котором качество на тесте окажется лучшим! 

Сгенерируем массив гиперпараметра, которые планируем перебрать:

In [179]:
alphas = np.linspace(0.01, 100, 10)
alphas

array([1.000e-02, 1.112e+01, 2.223e+01, 3.334e+01, 4.445e+01, 5.556e+01,
       6.667e+01, 7.778e+01, 8.889e+01, 1.000e+02])

Чтобы отобрать среди данного массива гиперпараметров лучший, воспользуйтесь конструкцией `GridSearchCV` из `sklearn`

In [180]:
from sklearn.model_selection import GridSearchCV

param_grid ={
    'LR__C': alphas
}

### Your code is here

search = GridSearchCV(pipe, param_grid, 
                      cv=custom_cv, scoring='accuracy')

search.fit(X_2, y_2)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)


Best parameter (CV score=0.63818):
{'LR__C': 0.01}


In [116]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

### Your code is here

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('LR', LogisticRegression(penalty='l2'))
])


In [117]:
from sklearn.model_selection import cross_validate

### Можно было и через KFold

custom_cv = [(X_2_train.index.to_list(), X_2_test.index.to_list())]


In [118]:
from sklearn.model_selection import GridSearchCV

alphas = np.arange(0, 100, 0.1)

param_grid ={
    'LR__C': alphas
}

search = GridSearchCV(pipe, param_grid, 
                      cv=custom_cv, scoring='accuracy')

search.fit(X_2, y_2)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

Best parameter (CV score=0.63818):
{'LR__C': 0.01}


In [120]:
pipe.set_params(LR__C=search.best_params_['LR__C'])

Pipeline(steps=[('scaler', StandardScaler()),
                ('LR', LogisticRegression(C=0.01))])

In [257]:
pipe.set_params(LR__C=0.002)

Pipeline(steps=[('scaler', StandardScaler()),
                ('LR', LogisticRegression(C=0.002))])

In [258]:
pipe.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()), ('LR', LogisticRegression(C=0.002))],
 'verbose': False,
 'scaler': StandardScaler(),
 'LR': LogisticRegression(C=0.002),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'LR__C': 0.002,
 'LR__class_weight': None,
 'LR__dual': False,
 'LR__fit_intercept': True,
 'LR__intercept_scaling': 1,
 'LR__l1_ratio': None,
 'LR__max_iter': 100,
 'LR__multi_class': 'auto',
 'LR__n_jobs': None,
 'LR__penalty': 'l2',
 'LR__random_state': None,
 'LR__solver': 'lbfgs',
 'LR__tol': 0.0001,
 'LR__verbose': 0,
 'LR__warm_start': False}

In [259]:
cv_result = cross_validate(pipe, X_2, y_2, cv=custom_cv, scoring='accuracy', return_train_score=True)

print(f"Accuracy на трейне: {np.mean(cv_result['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result['test_score']).round(3)}")

Accuracy на трейне: 0.638
Accuracy на тесте: 0.638


#### Задание 4: Бонус

Как вы знаете, подбор признаков является одной из самых важных частей машинного обучения. Сейчас вы попробуете на основе имеющихся признаков сгенерировать новые. В качестве новых признаков будем использовать мономы 2 степени. Можно использовать регуляризацию различного рода, выбор энкодера тоже за вами. Ваша задача - добиться качества `0.65` на тестовой выборке

In [189]:
data_2.describe()

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,Census_DeviceFamily_Windows.Server,Census_PrimaryDiskTypeName_NaN,Census_PrimaryDiskTypeName_SSD,Census_PrimaryDiskTypeName_UNKNOWN,Census_PrimaryDiskTypeName_Unspecified,Census_OSArchitecture_arm64,Census_OSArchitecture_x86,Census_GenuineStateName_IS_GENUINE,Census_GenuineStateName_OFFLINE,Census_GenuineStateName_UNKNOWN
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.500300,0.500300,0.500300,6.848175,0.017090,47867.326116,1.324916,1.021140,0.98768,107.989305,...,0.001590,0.001460,0.276885,0.040090,0.031505,0.000065,0.091405,0.883690,0.025830,0.001670
std,0.063468,0.054643,0.106269,1.014770,0.129607,13975.401102,0.520226,0.165952,0.11031,62.960237,...,0.039843,0.038182,0.447460,0.196171,0.174679,0.008062,0.288185,0.320597,0.158628,0.040832
min,0.000000,0.000000,0.000000,0.000000,0.000000,39.000000,1.000000,0.000000,0.00000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.493175,0.485433,0.466667,7.000000,0.000000,49480.000000,1.000000,1.000000,1.00000,51.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,0.493175,0.530941,0.512821,7.000000,0.000000,53447.000000,1.000000,1.000000,1.00000,97.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,0.552153,0.530941,0.552795,7.000000,0.000000,53447.000000,2.000000,1.000000,1.00000,160.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,8.000000,1.000000,70486.000000,5.000000,4.000000,1.00000,222.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [192]:
numeric_columns = numeric_columns.drop('HasDetections')

In [195]:
len(numeric_columns)

47

In [194]:
len(categorical_mean_columns)

19

In [198]:
categorical_columns = [col for col in categorical_columns if col not in categorical_mean_columns]

In [200]:
len(categorical_columns)

6

In [238]:
polinom_columns = []
polinom_columns.extend(numeric_columns)
polinom_columns.extend(categorical_mean_columns)

In [239]:
data_2[polinom_columns].describe()

,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,...,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_OSVersion,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_ActivationChannel,Census_FlightRing
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,6.848175,0.017090,47867.326116,1.324916,1.021140,0.98768,107.989305,81288.197287,24.870919,169.913125,...,0.500300,0.500300,0.500300,0.500300,0.500300,0.500300,0.500300,0.500300,0.500300,0.500300
std,1.014770,0.129607,13975.401102,0.520226,0.165952,0.11031,62.960237,48038.292735,4.661743,89.285504,...,0.035151,0.035495,0.042183,0.028112,0.027339,0.027266,0.022611,0.018534,0.022824,0.007153
min,0.000000,0.000000,39.000000,1.000000,0.000000,0.00000,1.000000,7.000000,1.000000,1.000000,...,0.000000,0.197531,0.000000,0.000000,0.250000,0.333333,0.444068,0.339623,0.338983,0.433476
25%,7.000000,0.000000,49480.000000,1.000000,1.000000,1.00000,51.000000,37761.000000,24.870919,89.000000,...,0.501220,0.502902,0.475493,0.464654,0.469357,0.469351,0.477921,0.478193,0.492479,0.501748
50%,7.000000,0.000000,53447.000000,1.000000,1.000000,1.00000,97.000000,81288.197287,24.870919,181.000000,...,0.501220,0.502902,0.502677,0.516403,0.513553,0.513910,0.482301,0.497938,0.492479,0.501748
75%,7.000000,0.000000,53447.000000,2.000000,1.000000,1.00000,160.000000,121351.250000,27.000000,267.000000,...,0.501220,0.502902,0.533063,0.524112,0.513553,0.513910,0.524273,0.517675,0.497147,0.501748
max,8.000000,1.000000,70486.000000,5.000000,4.000000,1.00000,222.000000,167962.000000,52.000000,296.000000,...,1.000000,0.525853,1.000000,0.524112,1.000000,1.000000,0.524273,0.517675,0.596689,0.511111


In [240]:
pol = []
for col in polinom_columns:
    if data_2[col].nunique() > 2:
        pol.append(col)
    else:
        print(f"{col} is bollean")

pol

IsSxsPassiveMode is bollean
HasTpm is bollean
AutoSampleOptIn is bollean
Census_HasOpticalDiskDrive is bollean
Census_IsPortableOperatingSystem is bollean
Census_IsSecureBootEnabled is bollean
Census_IsTouchEnabled is bollean
Census_IsPenCapable is bollean


['RtpStateBitfield',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'OsBuild',
 'OsSuite',
 'IsProtected',
 'SMode',
 'IeVerIdentifier',
 'Firewall',
 'UacLuaenable',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_PrimaryDiskTotalCapacity',
 'Census_SystemVolumeTotalCapacity',
 'Census_TotalPhysicalRAM',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_InternalBatteryNumberOfCharges',
 'Census_OSBuildNumber',
 'Census_OSBuildRevision',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSUILocaleIdentifier',
 'Census_IsFlightsDisabled',
 'Census_FirmwareManufacturerIdentifier',
 'Census_Firmwar

In [241]:
### Your code is here

data_3 = data_2.copy()

for col in pol:
    data_3[f"{col}^2"] = data_3[col] ** 2

In [242]:
data_3

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,Census_ChassisTypeName^2,Census_PowerPlatformRoleName^2,Census_OSVersion^2,Census_OSBranch^2,Census_OSEdition^2,Census_OSSkuName^2,Census_OSInstallTypeName^2,Census_OSWUAutoUpdateOptionsName^2,Census_ActivationChannel^2,Census_FlightRing^2
0,0.552153,0.530941,0.587952,7.0,0,53447.0,1.0,1.0,1,29,...,0.276326,0.276521,0.318499,0.274694,0.263737,0.264103,0.274863,0.247942,0.242536,0.251751
1,0.448751,0.528931,0.484434,7.0,0,53447.0,1.0,1.0,1,93,...,0.251221,0.252911,0.280058,0.274694,0.263737,0.264103,0.272558,0.247942,0.242536,0.245411
2,0.552153,0.530941,0.557522,7.0,0,53447.0,1.0,1.0,1,86,...,0.276326,0.276521,0.318499,0.274694,0.220296,0.220291,0.274863,0.267987,0.265801,0.251751
3,0.552153,0.530941,0.714442,7.0,0,53447.0,1.0,1.0,1,88,...,0.272964,0.276521,0.284156,0.274694,0.263737,0.264103,0.274863,0.267987,0.265801,0.251751
4,0.552153,0.530941,0.541395,7.0,0,53447.0,1.0,1.0,1,18,...,0.232829,0.252911,0.239343,0.274694,0.220296,0.220291,0.231133,0.267987,0.242536,0.251751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.552153,0.530941,0.545543,0.0,1,3371.0,2.0,1.0,1,160,...,0.251221,0.252911,0.261763,0.266672,0.220296,0.220291,0.274863,0.228669,0.247155,0.251751
199996,0.493175,0.530941,0.495413,7.0,0,53447.0,1.0,1.0,1,60,...,0.276326,0.131036,0.252684,0.266672,0.220296,0.220291,0.228408,0.225669,0.247155,0.251751
199997,0.341965,0.485433,0.407407,7.0,0,62773.0,1.0,1.0,1,9,...,0.276326,0.276521,0.256040,0.274694,0.284518,0.284498,0.274863,0.247942,0.247155,0.251751
199998,0.552153,0.460718,0.524096,7.0,0,49480.0,2.0,1.0,1,158,...,0.113763,0.131036,0.197019,0.226935,0.220296,0.220291,0.227962,0.247942,0.247155,0.251751


In [270]:
from sklearn.model_selection import train_test_split 

X = data_3.drop(columns="HasDetections")
Y = data_3['HasDetections']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)

In [271]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('LR', LogisticRegression(penalty='l2'))
])


In [272]:
from sklearn.model_selection import cross_validate

custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

In [269]:
alphas = np.arange(0, 1, 0.001)
alphas

array([0.   , 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008,
       0.009, 0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017,
       0.018, 0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026,
       0.027, 0.028, 0.029, 0.03 , 0.031, 0.032, 0.033, 0.034, 0.035,
       0.036, 0.037, 0.038, 0.039, 0.04 , 0.041, 0.042, 0.043, 0.044,
       0.045, 0.046, 0.047, 0.048, 0.049, 0.05 , 0.051, 0.052, 0.053,
       0.054, 0.055, 0.056, 0.057, 0.058, 0.059, 0.06 , 0.061, 0.062,
       0.063, 0.064, 0.065, 0.066, 0.067, 0.068, 0.069, 0.07 , 0.071,
       0.072, 0.073, 0.074, 0.075, 0.076, 0.077, 0.078, 0.079, 0.08 ,
       0.081, 0.082, 0.083, 0.084, 0.085, 0.086, 0.087, 0.088, 0.089,
       0.09 , 0.091, 0.092, 0.093, 0.094, 0.095, 0.096, 0.097, 0.098,
       0.099, 0.1  , 0.101, 0.102, 0.103, 0.104, 0.105, 0.106, 0.107,
       0.108, 0.109, 0.11 , 0.111, 0.112, 0.113, 0.114, 0.115, 0.116,
       0.117, 0.118, 0.119, 0.12 , 0.121, 0.122, 0.123, 0.124, 0.125,
       0.126, 0.127,

In [253]:
from sklearn.model_selection import GridSearchCV

param_grid ={
    'LR__C': alphas
}

### Your code is here

search = GridSearchCV(pipe, param_grid, 
                      cv=custom_cv, scoring='accuracy')

search.fit(X_2, y_2)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

E:\Рабочий стол С\python_projects\carpov_courses_python\machine_learning\less_10\venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 10000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "E:\Рабочий стол С\python_projects\carpov_courses_python\machine_learning\less_10\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "E:\Рабочий стол С\python_projects\carpov_courses_python\machine_learning\less_10\venv\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *a

Best parameter (CV score=0.63844):
{'LR__C': 0.002}


In [273]:
pipe.set_params(LR__C = search.best_params_['LR__C'])

Pipeline(steps=[('scaler', StandardScaler()),
                ('LR', LogisticRegression(C=0.002))])

In [274]:
pipe.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()), ('LR', LogisticRegression(C=0.002))],
 'verbose': False,
 'scaler': StandardScaler(),
 'LR': LogisticRegression(C=0.002),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'LR__C': 0.002,
 'LR__class_weight': None,
 'LR__dual': False,
 'LR__fit_intercept': True,
 'LR__intercept_scaling': 1,
 'LR__l1_ratio': None,
 'LR__max_iter': 100,
 'LR__multi_class': 'auto',
 'LR__n_jobs': None,
 'LR__penalty': 'l2',
 'LR__random_state': None,
 'LR__solver': 'lbfgs',
 'LR__tol': 0.0001,
 'LR__verbose': 0,
 'LR__warm_start': False}

In [275]:
cv_result_pipe = cross_validate(pipe, X, y, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)

print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

Accuracy на трейне: 0.639
Accuracy на тесте: 0.639


In [276]:
data_2

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,Census_DeviceFamily_Windows.Server,Census_PrimaryDiskTypeName_NaN,Census_PrimaryDiskTypeName_SSD,Census_PrimaryDiskTypeName_UNKNOWN,Census_PrimaryDiskTypeName_Unspecified,Census_OSArchitecture_arm64,Census_OSArchitecture_x86,Census_GenuineStateName_IS_GENUINE,Census_GenuineStateName_OFFLINE,Census_GenuineStateName_UNKNOWN
0,0.552153,0.530941,0.587952,7.0,0,53447.0,1.0,1.0,1,29,...,0,0,0,0,0,0,0,1,0,0
1,0.448751,0.528931,0.484434,7.0,0,53447.0,1.0,1.0,1,93,...,0,0,0,0,0,0,0,0,1,0
2,0.552153,0.530941,0.557522,7.0,0,53447.0,1.0,1.0,1,86,...,0,0,1,0,0,0,0,1,0,0
3,0.552153,0.530941,0.714442,7.0,0,53447.0,1.0,1.0,1,88,...,0,0,0,1,0,0,0,1,0,0
4,0.552153,0.530941,0.541395,7.0,0,53447.0,1.0,1.0,1,18,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.552153,0.530941,0.545543,0.0,1,3371.0,2.0,1.0,1,160,...,0,0,0,0,0,0,0,1,0,0
199996,0.493175,0.530941,0.495413,7.0,0,53447.0,1.0,1.0,1,60,...,0,0,1,0,0,0,0,1,0,0
199997,0.341965,0.485433,0.407407,7.0,0,62773.0,1.0,1.0,1,9,...,0,0,0,0,0,0,0,1,0,0
199998,0.552153,0.460718,0.524096,7.0,0,49480.0,2.0,1.0,1,158,...,0,0,1,0,0,0,1,1,0,0
